Extend the Dataset with Behavioral Variants

In [ ]:
!npm install -g javascript-obfuscator

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
changed 92 packages in 2s
⠴
⠴26 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [ ]:
from itertools import chain
import subprocess, tempfile, os

In [ ]:
!pip install js2py
!pip install evaluate
!pip install esprima

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25982 sha256=343971b00c110df89a94d5ba8311cdf82cf14383b23236e01fd91907c9b532bc
  Stored in directory: /root/.cache/pip/wheels/a5/9a/30/1003e89ab4555b81840ca46d361bf184f1e6ad880cae3b62a9
Successfully built pyjsparser
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for esprima: filename=esprima-4.0.1-py3-none-any.whl size=62243 sha256=7c922e65cac1e141fe957e5576f84d04c0446bfc7deb1e9d8e0b47d190808d07
  Stored in directory: /root/.cache/pip/wheels/ee/7c/26/04ddea9cd69d1b03b38932cca35cd118b730f13598d33a665d
Successfully built esprima


In [ ]:
import esprima
from datasets import Dataset
from transformers import RobertaTokenizerFast, AutoTokenizer, AutoConfig, PretrainedConfig, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM, GenerationConfig
import torch
from evaluate import load
import numpy as np
import matplotlib.pyplot as plt
import js2py
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import EarlyStoppingCallback
from evaluate import load
import json
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
full_dataset = load_dataset('json', data_files='converted_dataset.json', split='train')

NameError: name 'load_dataset' is not defined

In [ ]:
plain_path = '/content/drive/MyDrive/saved_functions'

def synthesize_obfuscations(js_code, num_variants=3):
    variants = []
    obf_count = 0
    for i in range(num_variants):
        with tempfile.NamedTemporaryFile(suffix='.js', mode='w+', delete=False) as f:
            f.write(js_code)
            f.flush()
            obfus_file = tempfile.NamedTemporaryFile(suffix=".js", delete=False)
            obfus_file.close()  # We'll just use its name
            # Run the obfuscator, directing output to the temp file
            print(f"Obfuscating {obf_count}")
            output = subprocess.run(
                ['javascript-obfuscator', f.name, '--compact', '--self-defending', '--output', obfus_file.name],
                capture_output=True, text=True
            )
            if output.returncode == 0:
                with open(obfus_file.name, 'r') as obf:
                    obfus_code = obf.read()
                    variants.append(obfus_code)
            obf_count += 1
    return variants

def load_js_pairs(plain_dir):
    plain_files = os.listdir(plain_dir)
    pairs = []
    count = 0
    for pf in plain_files:
        with open(os.path.join(plain_dir, pf), 'r') as plain_file:
                count += 1
                print(count)
                pairs.append(plain_file.read())
                if count >= 2:
                    break
    return pairs

js_data = load_js_pairs(plain_path)
extended_pairs = []
for plain_code in js_data:
    print(len(plain_code))
    #obfuscated_versions = synthesize_obfuscations(plain_code)
    #for variant in obfuscated_versions:
    #    extended_pairs.append((plain_code, variant))


1
2
445
1822


In [ ]:
print(extended_pairs)

[("function changeBg(){document.body.style.backgroundColor = '#' + Math.floor(Math.random() * 16777215).toString(16);}", "(function (_0x5b1ff9, _0x1fade3) {\n    var _0x27e92f = a0_0x5441, _0xc3a689 = _0x5b1ff9();\n    while (!![]) {\n        try {\n            var _0x5e2431 = -parseInt(_0x27e92f(0xd9)) / 0x1 + parseInt(_0x27e92f(0xd0)) / 0x2 + parseInt(_0x27e92f(0xd5)) / 0x3 * (parseInt(_0x27e92f(0xdb)) / 0x4) + parseInt(_0x27e92f(0xd4)) / 0x5 + parseInt(_0x27e92f(0xd1)) / 0x6 * (-parseInt(_0x27e92f(0xd8)) / 0x7) + parseInt(_0x27e92f(0xd6)) / 0x8 * (-parseInt(_0x27e92f(0xd3)) / 0x9) + parseInt(_0x27e92f(0xd2)) / 0xa;\n            if (_0x5e2431 === _0x1fade3)\n                break;\n            else\n                _0xc3a689['push'](_0xc3a689['shift']());\n        } catch (_0x495490) {\n            _0xc3a689['push'](_0xc3a689['shift']());\n        }\n    }\n}(a0_0x8349, 0xd39a4));\nfunction changeBg() {\n    var _0x38ceb4 = a0_0x5441;\n    document['body'][_0x38ceb4(0xcf)]['backgroun

Curriculum Learning

In [ ]:
def get_complexity_score(js_code):
    try:
        tree = esprima.parseScript(js_code, tolerant=True)
        def ast_depth(node):
            if not isinstance(node, dict): return 0
            return 1 + max([ast_depth(child) for key, child in node.items() if isinstance(child, (dict, list))] + [0])
        return ast_depth(tree.toDict()) + js_code.count('\n')
    except:
        return 0  # fallback for failed parse

levels = {'easy': [], 'medium': [], 'hard': []}
for plain, obfus in full_dataset:
    score = get_complexity_score(plain)
    if score < 20:
        levels['easy'].append((plain, obfus))
    elif score < 50:
        levels['medium'].append((plain, obfus))
    else:
        levels['hard'].append((plain, obfus))

In [ ]:
def test_functional_match(code1, code2):
    try:
        result1 = js2py.eval_js(code1)
        result2 = js2py.eval_js(code2)
        return result1 == result2
    except:
        return False

#DOES NOT WORK
tokenizer = RobertaTokenizerFast.from_pretrained("Salesforce/codet5-large")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-large")
equiv_easy = 0
equiv_medium = 0
equiv_hard = 0
bleu_easy = 0
bleu_medium = 0
bleu_hard = 0


def tokenize(example):
    return tokenizer(
        example['input'],
        text_target=example['output'],
        padding="max_length",
        truncation=True,
        max_length=1024
    )

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

In [ ]:
print(tokenizer.model_max_length)

512


In [ ]:
#TRAIN FOR EASY
level_name = 'easy'
data_pairs = levels[level_name]
print("Starting training for " + level_name + "\n")


# Split data into train and validation
train_pairs, val_pairs = train_test_split(data_pairs, test_size=0.1, random_state=42)

# Convert to Hugging Face datasets
train_dict = {
    'input': [x[0] for x in train_pairs],
    'output': [x[1] for x in train_pairs]
}
val_dict = {
    'input': [x[0] for x in val_pairs],
    'output': [x[1] for x in val_pairs]
}

train_dataset = Dataset.from_dict(train_dict).map(tokenize, batched=True)
val_dataset = Dataset.from_dict(val_dict).map(tokenize, batched=True)

# Training arguments with evaluation enabled
args = Seq2SeqTrainingArguments(
    output_dir=f"./code_obfuscator_{level_name}",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="./logs",
    save_total_limit=2,
    eval_strategy='epoch',        # <-- This triggers eval each epoch
    logging_strategy='epoch',           # Optional: logs every epoch
    save_strategy='epoch',              # Optional: save every epoch
    load_best_model_at_end=True,        # Optional: load best based on val loss
    metric_for_best_model='loss',       # Select model based on val loss
    generation_max_length=1024,
    weight_decay=0.01
)

# Define trainer with eval dataset
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,            # <-- Add validation set here
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

start = time.time()
trainer.train()
end = time.time()

print(f"Training Time: {end - start:.2f} seconds")


Starting training for easy



Map:   0%|          | 0/20957 [00:00<?, ? examples/s]

Map:   0%|          | 0/2329 [00:00<?, ? examples/s]

/tmp/ipython-input-16-3652713484.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.817000,0.723694
2,0.730400,0.710906
3,0.716100,0.703649
4,0.708100,0.700998
5,0.702500,0.698422
6,0.698100,0.697403
7,0.694600,0.695699
8,0.691700,0.695615
9,0.689100,0.695042
10,0.687300,0.695308


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Training Time: 37358.95 seconds


In [ ]:


# BLEU metric => DOES NOT MAKE SENSE SO IGNORE
bleu = load("bleu")

# Define evaluation set size
test_size = min(50, len(val_dataset))
print(f"Timing predictions for {test_size} samples...\n")

# Get the evaluation subset
test_data = val_dataset.select(range(test_size))

# Store decoded values
decoded_preds = []
decoded_labels = []
decoded_inputs = []
timings = []

for i in tqdm(range(test_size)):
    start = time.time()
    pred = trainer.predict(test_data.select([i]))
    end = time.time()

    timings.append(end - start)

    decoded_pred = tokenizer.decode(pred.predictions[0], skip_special_tokens=True)
    decoded_label = tokenizer.decode(pred.label_ids[0], skip_special_tokens=True)
    decoded_input = tokenizer.decode(test_data[i]["input_ids"], skip_special_tokens=True)

    print(len(decoded_pred), decoded_pred[:100])  # Log prediction length

    decoded_preds.append(decoded_pred)
    decoded_labels.append(decoded_label)
    decoded_inputs.append(decoded_input)


# # Compute BLEU
# bleu_result = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
# print(f"\nBLEU Score: {bleu_result['bleu']:.4f}")
# print(f"Total prediction time: {sum(timings):.2f} seconds")
# print(f"Average time per sample: {sum(timings)/test_size:.4f} seconds")

# Save predictions
with open("predictions.jsonl", "w", encoding="utf-8") as f:
    for i, (inp, pred, label, t) in enumerate(zip(decoded_inputs, decoded_preds, decoded_labels, timings)):
        json.dump({
            "id": i,
            "input": inp,
            "prediction": pred,
            "label": label,
            "time_seconds": round(t, 4)
        }, f)
        f.write("\n")

# # Save BLEU per level
# if level_name == "easy":
#     bleu_easy = bleu_result["bleu"]
# elif level_name == "medium":
#     bleu_medium = bleu_result["bleu"]
# elif level_name == "hard":
#     bleu_hard = bleu_result["bleu"]

# Download file (Colab)
files.download("predictions.jsonl")

losses = []
steps = []

logs = trainer.state.log_history  # This contains training logs
for entry in logs:
    if "loss" in entry and "step" in entry:
        losses.append(entry["loss"])
        steps.append(entry["step"])

plt.plot(steps, losses)
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.title(f"Loss Curve ({level_name.capitalize()} Level)")
plt.grid(True)
plt.show()

# Optionally print some predictions
print("\nExample Predictions:\n")
for i in range(min(3, test_size)):
    print(f"Input {i+1}: {decoded_inputs[i]}")
    print(f"Prediction {i+1}: {decoded_preds[i]}")
    print(f"Label {i+1}: {decoded_labels[i]}")
    print(f"Time: {timings[i]:.4f} sec\n")


Timing predictions for 50 samples...



  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:00<00:24,  2.01it/s]

  4%|▍         | 2/50 [00:00<00:23,  2.05it/s]

  6%|▌         | 3/50 [00:01<00:22,  2.07it/s]

...

100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


BLEU Score: 0.0000
Total prediction time: 23.13 seconds
Average time per sample: 0.4625 seconds


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NameError: name 'logs' is not defined

In [ ]:
def train_on_level(level_name, data_pairs):
  print("Starting training for " + level_name + "\n")

  # Split data into train and validation
  train_pairs, val_pairs = train_test_split(data_pairs, test_size=0.1, random_state=42)

  # Convert to Hugging Face datasets
  train_dict = {
      'input': [x[0] for x in train_pairs],
      'output': [x[1] for x in train_pairs]
  }
  val_dict = {
      'input': [x[0] for x in val_pairs],
      'output': [x[1] for x in val_pairs]
  }

  train_dataset = Dataset.from_dict(train_dict).map(tokenize, batched=True)
  val_dataset = Dataset.from_dict(val_dict).map(tokenize, batched=True)

  # Define trainer with eval dataset
  trainer = Seq2SeqTrainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,            # <-- Add validation set here
      tokenizer=tokenizer,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
  )

  start = time.time()
  trainer.train()
  end = time.time()

  print(f"Training Time: {end - start:.2f} seconds")
 # bleu = load("bleu")

  # Define evaluation set size
test_size = min(50, len(val_dataset))
print(f"Timing predictions for {test_size} samples...\n")

# Get the evaluation subset
test_data = val_dataset.select(range(test_size))

# Store decoded values
decoded_preds = []
decoded_labels = []
decoded_inputs = []
timings = []

# Predict each sample individually with timing
for i in tqdm(range(test_size)):
    start = time.time()
    pred = trainer.predict(test_data.select([i]))
    end = time.time()

    timings.append(end - start)

    decoded_pred = tokenizer.decode(pred.predictions[0], skip_special_tokens=True)
    decoded_label = tokenizer.decode(pred.label_ids[0], skip_special_tokens=True)
    decoded_input = tokenizer.decode(test_data[i]["input_ids"], skip_special_tokens=True)

    decoded_preds.append(decoded_pred)
    decoded_labels.append(decoded_label)
    decoded_inputs.append(decoded_input)

    # # Compute BLEU
    # bleu_result = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    # print(f"\nBLEU Score: {bleu_result['bleu']:.4f}")
    # print(f"Total prediction time: {sum(timings):.2f} seconds")
    # print(f"Average time per sample: {sum(timings)/test_size:.4f} seconds")

    # Save predictions
    with open("predictions.jsonl", "w", encoding="utf-8") as f:
        for i, (inp, pred, label, t) in enumerate(zip(decoded_inputs, decoded_preds, decoded_labels, timings)):
            json.dump({
                "id": i,
                "input": inp,
                "prediction": pred,
                "label": label,
                "time_seconds": round(t, 4)
            }, f)
            f.write("\n")

    # # Save BLEU per level
    # if level_name == "easy":
    #     bleu_easy = bleu_result["bleu"]
    # elif level_name == "medium":
    #     bleu_medium = bleu_result["bleu"]
    # elif level_name == "hard":
    #     bleu_hard = bleu_result["bleu"]

    # Download file (Colab)
    files.download("predictions.jsonl")

    # Plot training loss
    logs = trainer.state.log_history
    losses = [entry["loss"] for entry in logs if "loss" in entry]
    steps = [entry["step"] for entry in logs if "step" in entry]

    plt.plot(steps, losses)
    plt.xlabel("Training Steps")
    plt.ylabel("Loss")
    plt.title(f"Loss Curve ({level_name.capitalize()} Level)")
    plt.grid(True)
    plt.show()

    # Optionally print some predictions
    print("\nExample Predictions:\n")
    for i in range(min(3, test_size)):
        print(f"Input {i+1}: {decoded_inputs[i]}")
        print(f"Prediction {i+1}: {decoded_preds[i]}")
        print(f"Label {i+1}: {decoded_labels[i]}")
        print(f"Time: {timings[i]:.4f} sec\n")


# Stage-wise curriculum loop
for level in ['medium', 'hard']:
    train_on_level(level, levels[level])

# difficulty = ['Easy', 'Medium', 'Hard']
# bleu_scores = [bleu_easy, bleu_medium, bleu_hard]

# plt.plot(difficulty, bleu_scores, marker='o')
# plt.title("BLEU Score by Curriculum Level")
# plt.ylabel("Score")
# plt.show()

Starting training for easy



Map:   0%|          | 0/303 [00:00<?, ? examples/s]

/tmp/ipython-input-12-2725210815.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
500,1.053900
1000,0.710800
1500,0.629700
2000,0.555000


Training Time: 659.11 seconds
Timing for 50:


KeyError: 0